In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../../')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from src.data_loader import DataLoader
from src.data_loader import load_dataset
from src.forecasting import TimeSeriesForecaster
from src.counterfactuals import CounterfactualGenerator

In [3]:
data_path = "../../data/"
lst_arrays, lst_arrays_test = load_dataset('simulated', data_path)

CHANGE_COLS = [0, 1, 2]
TARGET_COL = 3

back_horizon, horizon = 10, 5

dataset = DataLoader(horizon, back_horizon)
dataset.preprocessing(
    lst_train_arrays=lst_arrays,
    lst_test_arrays=lst_arrays_test,
    train_size=0.8,
    normalize=True,
    sequence_stride=horizon,
    target_col=TARGET_COL,
)

print(dataset.X_train.shape, dataset.Y_train.shape)
print(dataset.X_val.shape, dataset.Y_val.shape)
print(dataset.X_test.shape, dataset.Y_test.shape)

(125, 10, 3) (125, 5, 1)
(29, 10, 3) (29, 5, 1)
(37, 10, 3) (37, 5, 1)


In [4]:
gru_forecaster = TimeSeriesForecaster()
gru_forecaster.train_model('gru', dataset, back_horizon, horizon)



Epoch 1/200

2/2 [==============================] - 3s 560ms/step - loss: 0.4892 - val_loss: 0.5229
Epoch 2/200
2/2 [==============================] - 0s 36ms/step - loss: 0.4779 - val_loss: 0.5113
Epoch 3/200
2/2 [==============================] - 0s 37ms/step - loss: 0.4665 - val_loss: 0.4997
Epoch 4/200
2/2 [==============================] - 0s 36ms/step - loss: 0.4551 - val_loss: 0.4881
Epoch 5/200
2/2 [==============================] - 0s 35ms/step - loss: 0.4435 - val_loss: 0.4763
Epoch 6/200
2/2 [==============================] - 0s 36ms/step - loss: 0.4320 - val_loss: 0.4644
Epoch 7/200
2/2 [==============================] - 0s 36ms/step - loss: 0.4204 - val_loss: 0.4524
Epoch 8/200
2/2 [==============================] - 0s 39ms/step - loss: 0.4089 - val_loss: 0.4402
Epoch 9/200
2/2 [==============================] - 0s 41ms/step - loss: 0.3974 - val_loss: 0.4280
Epoch 10/200
2/2 [==============================] - 0s 35ms/step - loss: 0.3859 - val_loss: 0.4160
Epoch 11/200
2/

In [6]:
cf_generator = CounterfactualGenerator(dataset, gru_forecaster, TARGET_COL, CHANGE_COLS, 42, horizon)
cf_generator.generate_counterfactuals('gru')

Generating CFs for 37 samples in total...
Desired center 0.5 in 10 timesteps.
Desired center: 0.5; after scaling: 0.5727
Desired center: 0.5; after scaling: 0.5727
Desired center: 0.5; after scaling: 0.5727
Desired center: 0.5; after scaling: 0.5727
Desired center: 0.5; after scaling: 0.5727
Desired center: 0.5; after scaling: 0.5727
Desired center: 0.5; after scaling: 0.5727
Desired center: 0.5; after scaling: 0.5727
Desired center: 0.5; after scaling: 0.5727
Desired center: 0.5; after scaling: 0.5727
Desired center: 0.5; after scaling: 0.5727
Desired center: 0.5; after scaling: 0.5727
Desired center: 0.5; after scaling: 0.5727
Desired center: 0.5; after scaling: 0.5727
Desired center: 0.5; after scaling: 0.5727
Desired center: 0.5; after scaling: 0.5727
Desired center: 0.5; after scaling: 0.5727
Desired center: 0.5; after scaling: 0.5727
Desired center: 0.5; after scaling: 0.5727
Desired center: 0.5; after scaling: 0.5727
Desired center: 0.5; after scaling: 0.5727
Desired center: 0.5

d:\Dokumentumok\BME\PhD\codes\ts_counterfactuals\CausalConstrainedCFTS\CausalConstrainedCFTS\notebooks\forecasting\../..\src\evaluation_metrics.py:130: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method="backfill", axis=1)
